In [49]:
import pystan
import numpy as np
import pickle

model = """
data {
    int N;
    int M;
    real<lower=0>  V[N,M];
}
parameters {
    //real<lower=0> lambda;
    real<lower=0> W[N,2];
    real<lower=0> H[2,M];
    real eps;
}
model {
    eps ~ normal(0,1);
    for (i in 1:N)
        for (j in 1:2)
            W[i,j] ~ exponential(0.5);
        
    for (i in 1:2)
        for (j in 1:M)
            H[i,j] ~ exponential(0.5);
    
    for (i in 1:N)
        for (j in 1:M)
            V[i,j] ~ normal(W[i,1]*H[1,j]+W[i,2]*H[2,j], eps);
}
"""
data = np.matrix([[36, 3, 45, 54], \
                            [4, 34, 23, 31], \
                            [9, 65, 11, 0], \
                            [17, 3, 3, 0], \
                            [0, 14, 7, 4]])

In [53]:
data_dict = {'N': data.shape[0], 'M': data.shape[1] ,'V' : data}
#fit_eps = pystan.stan(model_code=model, data=data_dict, iter=1000, chains=1)
print(fit_eps)

Inference for Stan model: anon_model_3af51e32c9a82cadff167620130feac4.
1 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=500.

         mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
W[0,0]   9.84    0.21   2.71   5.72   7.79   9.34  11.51  16.35    164    1.0
W[1,0]    4.0    0.09   1.24   2.19   3.09   3.89   4.66   7.09    193    1.0
W[2,0]   0.46    0.02    0.4   0.01   0.15   0.36   0.65   1.45    357    1.0
W[3,0]    0.9    0.04   0.54   0.07    0.5   0.84   1.24   2.15    179    1.0
W[4,0]    0.7    0.03   0.44   0.07   0.38   0.62   0.93   1.71    301   1.01
W[0,1]    0.4    0.02   0.36 9.0e-3   0.13   0.31   0.58   1.36    500    1.0
W[1,1]   3.87     0.1   1.26   2.05   2.94   3.61   4.62   6.66    166    1.0
W[2,1]    7.8    0.17    2.3   4.55   6.11   7.47   9.04   13.2    176   1.01
W[3,1]   0.56    0.02    0.4   0.05   0.27   0.48   0.77   1.59    500    1.0
W[4,1]    1.6    0.05   0.72    0.

In [48]:
len(fit.extract()['H'])

500

In [74]:
W = fit_eps.extract(['W'])

fit_eps.plot(W)

TypeError: iteration over a 0-d array

In [22]:
sm = pystan.StanModel(model_code=model)

#with open('model.pkl', 'wb') as f:

op = sm.optimizing(data=data_dict)
print(op)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_157ebfa8e4d63a708b8a68979b07955a NOW.


OrderedDict([('W', array([[ 0.15348321,  9.05577542],
       [ 3.35713715,  3.81300791],
       [ 6.6267781 ,  0.32614392],
       [ 0.29249375,  0.97160007],
       [ 1.43342333,  0.6036792 ]])), ('H', array([[ 0.35078532,  9.81816608,  1.42797337,  0.16807357],
       [ 3.61826211,  0.17755765,  4.90829185,  6.19188884]]))])


In [28]:
np.matmul(op['W'], op['H'])

array([[ 32.8200087 ,   3.1148458 ,  44.66755859,  56.09815125],
       [ 14.97409645,  33.63795879,  23.50925809,  24.17396716],
       [  3.50465066,  65.12071734,  11.06367224,   3.13323319],
       [  3.61810623,   3.04426726,   5.18656999,   6.0652001 ],
       [  2.68709342,  14.18077617,   5.00992403,   3.97883507]])

In [ ]:
np.mat